<a href="https://colab.research.google.com/github/Faraz011/Assignment-AIML-Lab/blob/main/Dynamic_Pricing_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

